In [5]:
import torch
import random
import numpy as np
import regex

RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
def split_train_val_test(df, props=[.8, .1, .1]):
    assert round(sum(props), 2) == 1 and len(props) >= 2
    train_df, test_df, val_df = None, None, None

    train_size = int(props[0] * len(df))
    val_size =  train_size + int(props[1] * len(df))
    test_size =val_size + int(props[2] * len(df)) 
    train_df = df.iloc[0:train_size]
    val_df = df.iloc[train_size:val_size]
    test_df = df.iloc[val_size:test_size]
    
    return train_df, val_df, test_df

In [7]:
import gensim.downloader as api

def download_embeddings(fasttetxt):
    # https://fasttext.cc/docs/en/english-vectors.html
    if fasttetxt:
      wv = api.load("fasttext-wiki-news-subwords-300")
    else:
      
      wv = api.load("word2vec-google-news-300")
      print("\nLoading complete!\n" +
            "Vocabulary size: {}".format(len(wv.vocab)))
    return wv


In [8]:
# Opening and preprocessing input file
import gensim.models
import pandas as pd
import nltk
nltk.download('punkt')
from tqdm import tqdm
from preprocess import clean_text

data = pd.read_csv('train.csv', quotechar='"')
data.sample(frac=1)


# to convert authors into numbers
author_to_number = {
    'EAP': 0,
    'HPL': 1,
    'MWS': 2
    
}

# lowercase, removing punctuation and tookenize sentences. Converting labels to int
training_text = ""
for i in range(len(data)):

    data['text'][i] = nltk.word_tokenize(regex.sub(r'[^\w\s]', '',data['text'][i].lower()))
    data['author'][i] = author_to_number[data['author'][i]]

print(data[0:10])
print(len(data))

from dataset import *

# Splitting dataset and generating vocab
train_df, val_df, test_df = split_train_val_test(data)
train_vocab, reversed_vocab = generate_vocab_map(train_df)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
        id                                               text author
0  id26305  [this, process, however, afforded, me, no, mea...      0
1  id17569  [it, never, once, occurred, to, me, that, the,...      1
2  id11008  [in, his, left, hand, was, a, gold, snuff, box...      0
3  id27763  [how, lovely, is, spring, as, we, looked, from...      2
4  id12958  [finding, nothing, else, not, even, gold, the,...      1
5  id22965  [a, youth, passed, in, solitude, my, best, yea...      2
6  id09674  [the, astronomer, perhaps, at, this, point, to...      0
7  id13515  [the, surcingle, hung, in, ribands, from, my, ...      0
8  id19322  [i, knew, that, you, could, not, say, to, your...      0
9  id00912  [i, confess, that, neither, the, structure, of...      2
19579


In [9]:
DOWNLOAD = False
# Use fastext or word2vec
FASTTEXT = False
WINDOW_SIZE = 5

EMBEDDING_DIM = 300
HIDDEN_DIM = 128
NUM_LAYERS = 1
BIDIRECTIONAL = True


In [10]:
# Downloading or generating word2vec embeddings

if DOWNLOAD:
    model = download_embeddings(FASTTEXT)
else:
    if FASTTEXT:
        model = gensim.models.FastText(sentences=train_df['text'], size=EMBEDDING_DIM, window=WINDOW_SIZE)
    else:
        model = gensim.models.Word2Vec(sentences=train_df['text'], size=EMBEDDING_DIM, window=WINDOW_SIZE)
                        

In [11]:
from dataset import HeadlineDataset
from torch.utils.data import RandomSampler

train_dataset = HeadlineDataset(train_vocab, train_df,model.wv, FASTTEXT)
val_dataset = HeadlineDataset(train_vocab, val_df,model.wv, FASTTEXT)
test_dataset = HeadlineDataset(train_vocab, test_df,model.wv, FASTTEXT)

# Now that we're wrapping our dataframes in PyTorch datsets, we can make use of PyTorch Random Samplers.
train_sampler = RandomSampler(train_dataset)
val_sampler = RandomSampler(val_dataset)
test_sampler = RandomSampler(test_dataset)

In [12]:
from torch.utils.data import DataLoader
from dataset import collate_fn
BATCH_SIZE = 16
train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler, collate_fn=collate_fn)
val_iterator = DataLoader(val_dataset, batch_size=BATCH_SIZE, sampler=val_sampler, collate_fn=collate_fn)
test_iterator = DataLoader(test_dataset, batch_size=BATCH_SIZE, sampler=test_sampler, collate_fn=collate_fn)

for x, y in test_iterator:
    print(x,y)
    break

tensor([[[ 0.2023,  0.3327, -0.2675,  ..., -0.2955, -0.1995, -0.6266],
         [ 0.0498, -0.1170, -0.1079,  ..., -0.0727, -0.0963, -0.2080],
         [ 0.1343, -0.3369, -0.3732,  ..., -0.2670, -0.5220, -0.7150],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.1035, -0.9070, -0.1016,  ..., -0.4207,  0.2501, -0.0609],
         [ 0.0454, -0.2868, -0.1376,  ..., -0.0651,  0.0015, -0.1849],
         [ 0.0106, -0.2581, -0.0899,  ...,  0.1391, -0.0131, -0.1398],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.2933, -0.6308,  0.0843,  ..., -0.3770,  0.3463,  0.1079],
         [-0.4181, -0.2825, -0.1756,  ...,  0

/content/dataset.py:161: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tokenized_word_tensor = torch.Tensor(tmp)


### Modeling

In [13]:
from models import ClassificationModel

model = ClassificationModel(len(train_vocab),embedding_dim=EMBEDDING_DIM,hidden_dim = HIDDEN_DIM,num_layers = NUM_LAYERS,bidirectional = BIDIRECTIONAL)

model.to(device)

ClassificationModel(
  (LSTM): LSTM(300, 128, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=256, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)

In the following cell, **instantiate the model with some hyperparameters, and select an appropriate loss function and optimizer.** 

Hint: we already use sigmoid in our model. What loss functions are availible for binary classification? Feel free to look at PyTorch docs for help!

In [14]:
from torch.optim import AdamW

criterion, optimizer = torch.nn.CrossEntropyLoss(), torch.optim.Adam(model.parameters(), lr=0.001)

### Part 3: Training and Evaluation [10 Points]
The final part of this HW involves training the model, and evaluating it at each epoch. **Fill out the train and test loops below.**

In [15]:
# returns the total loss calculated from criterion
def train_loop(model, criterion, iterator):
    model.train()
    total_loss = 0
    
    for x, y in tqdm(iterator):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()

        prediction = model(x)
        prediction = torch.squeeze(prediction)
        # y = y.round()
        y = y.long()
        

 
        loss = criterion(prediction,y)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    return total_loss

# returns:
# - true: a Python boolean array of all the ground truth values 
#         taken from the dataset iterator
# - pred: a Python boolean array of all model predictions. 
def val_loop(model, criterion, iterator):
    true, pred = [], []
    for x, y in tqdm(iterator):
        x = x.to(device)
        y = y.to(device)
    
        preds = model(x)
        preds.to(device)
        preds = torch.squeeze(preds)
        for i_batch in range(len(y)):
            true.append(y[i_batch])
            pred.append(torch.argmax(preds[i_batch]))
            
    return true, pred


In [16]:
# To test your eval implementation, let's see how well the untrained model does on our dev dataset.
# It should do pretty poorly.
from sklearn.metrics import f1_score, accuracy_score

from eval_utils import binary_macro_f1, accuracy
true, pred = val_loop(model, criterion, val_iterator)
true = [x.item() for x in true]
pred = [x.item() for x in pred]
print(f1_score(true, pred, average='weighted'))
print(accuracy_score(true, pred))


100%|██████████| 123/123 [00:04<00:00, 27.21it/s]


0.26611359256628586
0.32447623914154317


### Actually training the model

In [17]:
TOTAL_EPOCHS = 10
for epoch in range(TOTAL_EPOCHS):
    train_loss = train_loop(model, criterion, train_iterator)
    true, pred = val_loop(model, criterion, val_iterator)
    true = [x.item() for x in true]
    pred = [x.item() for x in pred]
    print(f"EPOCH: {epoch}")
    print(f"TRAIN LOSS: {train_loss}")
    print(f"VAL F-1: {f1_score(true, pred, average='weighted')}")
    print(f"VAL ACC: {accuracy_score(true, pred)}")


100%|██████████| 123/123 [00:04<00:00, 27.34it/s]


EPOCH: 0
TRAIN LOSS: 985.7116901874542
VAL F-1: 0.5715518445600153
VAL ACC: 0.5738375063873276


100%|██████████| 123/123 [00:04<00:00, 27.91it/s]


EPOCH: 1
TRAIN LOSS: 884.940908908844
VAL F-1: 0.5861754109844795
VAL ACC: 0.5896780786918753


100%|██████████| 123/123 [00:04<00:00, 27.65it/s]


...

EPOCH: 7
TRAIN LOSS: 644.8398031592369
VAL F-1: 0.6672287455535938
VAL ACC: 0.6668369954011242


100%|██████████| 123/123 [00:04<00:00, 27.89it/s]


EPOCH: 8
TRAIN LOSS: 605.1051699966192
VAL F-1: 0.6704467302558326
VAL ACC: 0.6709248850281042


100%|██████████| 123/123 [00:04<00:00, 27.70it/s]

EPOCH: 9
TRAIN LOSS: 561.894759863615
VAL F-1: 0.6641035162791222
VAL ACC: 0.6663260091977516


We can also look at the models performance on the held-out test set, using the same val_loop we wrote earlier.

In [18]:
true, pred = val_loop(model, criterion, test_iterator)
true = [x.item() for x in true]
pred = [x.item() for x in pred]
print(f"TEST F-1: {f1_score(true, pred, average='weighted')}")
print(f"TEST ACC: {accuracy_score(true, pred)}")

100%|██████████| 123/123 [00:04<00:00, 28.00it/s]


TEST F-1: 0.6521270583403529
TEST ACC: 0.6535513541134389
